<a href="https://nb.bohrium.dp.tech/detail/18475433825" target="_blank"><img src="https://cdn.dp.tech/bohrium/web/static/images/open-in-bohrium.svg" alt="Open In Bohrium"/></a>

<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-bottom:1rem">
    <p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
        ©️ <b><i>Copyright 2025 @ Authors</i></b><br/>
        <i>Authors: <a style="font-weight:bold" href="mailto:zhangd@dp.tch">Duo Zhang📨   </b></a></i><br/>
        <i>Date: 2025-06-10</i><br/>
        <i>License: <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"> Attribution-NonCommercial-ShareAlike 4.0 International</a></i><br/>
        <i>Quick start: You can click on the blue button at the top of the page <span style="background-color:rgb(85, 91, 228); color:white; padding: 3px; border-radius: 5px;box-shadow: 2px 2px 3px rgba(0, 0, 0, 0.3); font-size:0.75rem;"> Connect </span> , select<span style='color:rgb(85,91,228); font-weight:bold'> `registry.dp.tech/dptech/deepmd-kit:3.0.1-cuda12.1` </span>image and <span style='color:rgb(85,91,228); font-weight:bold'>`c12_m92_1 * NVIDIA V100`</span> machine, and at the same time mount the <span style='color:rgb(85,91,228); font-weight:bold'>`dpa3_model(v2)`</span> dataset, and wait for a few moments to run.</i><br/>
    </p>
</div>

# Introduction

**DPA3** is a multi-layer graph neural network founded on line graph series (LiGS), designed explicitly for the era of [large atomistic models (LAMs)](https://www.aissquare.com/openlam). The generalization error of the DPA3 model adheres to the scaling law. The scalability in the number of model parameters is attained by stacking additional layers within DPA3. Additionally, the model employs a dataset encoding mechanism that decouples the scaling of training data size from the model size within its multi-task training framework.

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/8ca7545d-d0f6-4952-9d6a-fc615a950f9e.png" alt="" width="600" style="display: block; margin: auto;">

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/c5adcd1b-0721-400b-a7c9-c8e4f25a78ec.png" alt="" width="600" style="display: block; margin: auto;">

When trained as problem-oriented potential energy models, the DPA3 model exhibits superior accuracy in the majority of benchmark cases, encompassing systems with diverse features, including molecules, bulk materials, surface and cluster catalysts, two-dimensional materials, and battery materials. When trained as a LAM on the OpenLAM-v1 dataset, the DPA-3.1-3M model exhibits state-of-the-art performance in the LAMBench benchmark suite for LAMs, demonstrating lowest overall zero-shot generalization error across 17 downstream tasks from a broad spectrum of research domains. This performance suggests superior accuracy as an out-of-the-box potential model, requiring minimal fine-tuning data for downstream scientific applications.

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/79ee197d-4283-4403-b3fe-5b0758e992a8.png" alt="" width="600" style="display: block; margin: auto;">



<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/0ab27a7f-a35b-463c-8606-e740a8d0d328.png" alt="" width="600" style="display: block; margin: auto;">


Reference: 

[1] Zhang, D., Peng, A., Cai, C., Li, W., Zhou, Y., Zeng, J., ... & Wang, H. (2025). Graph neural network model for the era of large atomistic models. [arXiv preprint arXiv:2506.01686](https://arxiv.org/abs/2506.01686).


**This notebook will detail the steps to train DPA3 model from scratch, finetune a pretrained model, perfrom python inference and run MD using ASE calculator.** 

**Note that this notebook will mainly focus on DPA3 model training and finetuning. For data preparation and other basic usages of DeePMD-kit, please refer to [this notebook](https://bohrium.dp.tech/notebooks/71979198691) first.**

## 1. Installation

We install DeePMD-kit v3.1.0 to run DPA3.

In [1]:
!git clone https://github.com/deepmodeling/deepmd-kit.git -b v3.1.0rc0

Cloning into 'deepmd-kit'...
remote: Enumerating objects: 39775, done.
remote: Counting objects: 100% (933/933), done.
remote: Compressing objects: 100% (519/519), done.
remote: Total 39775 (delta 708), reused 422 (delta 414), pack-reused 38842 (from 3)
Receiving objects: 100% (39775/39775), 64.52 MiB | 4.85 MiB/s, done.
Resolving deltas: 100% (29723/29723), done.
Note: switching to '265d09427c4be1a0f2466e6bd9789bed2e59caa0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [2]:
!cd deepmd-kit && pip install -e .

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///deepmd-kit
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for deepmd-kit (pyproject.toml) ... done
  Created wheel for deepmd-kit: filename=deepmd_kit-3.1.0rc0-py37-none-linux_x86_64.whl size=525291 sha256=acdf35f3586adec8115f6f206bca214a9516f78d0f93bddaeeedf87fbc75ea16
  Stored in directory: /tmp/pip-ephem-wheel-cache-wzqrx442/wheels/58/8d/ec/84fa7b35fd22f7a8dd31c1e771b93e731fdce060d59c52ab78
Successfully built deepmd-kit
  Attempting uninstall: deepmd-kit
    Found existing installation: deepmd-kit 3.0.1
    Uninstalling deepmd-kit-3.0.1:
      Successfully uninstalled deepmd-kit-3.0.1


In [3]:
!dp --version

DeePMD-kit v3.1.0rc0


## 2. Train the DPA3 model from scratch

We will first train a 6-layer DPA3 model from scratch on the example water system.

In [4]:
!cd /deepmd-kit/examples/water/dpa3 && cat input_torch.json

{
  "_comment": "that's all",
  "model": {
    "type_map": [
      "O",
      "H"
    ],
    "descriptor": {
      "type": "dpa3",
      "repflow": {
        "n_dim": 128,
        "e_dim": 64,
        "a_dim": 32,
        "nlayers": 6,
        "e_rcut": 6.0,
        "e_rcut_smth": 5.3,
        "e_sel": 120,
        "a_rcut": 4.0,
        "a_rcut_smth": 3.5,
        "a_sel": 30,
        "axis_neuron": 4,
        "fix_stat_std": 0.3,
        "a_compress_rate": 1,
        "a_compress_e_rate": 2,
        "a_compress_use_split": true,
        "update_angle": true,
        "smooth_edge_update": true,
        "edge_init_use_dist": true,
        "use_exp_switch": true,
        "update_style": "res_residual",
        "update_residual": 0.1,
        "update_residual_init": "const"
      },
      "activation_function": "silut:10.0",
      "use_tebd_bias": false,
      "precision": "float32",
      "concat_output_tebd": false
    },
    "fitting_net": {
      "neuron": [
        240,
        240,


The definitions of params in DPA3 desciptor can be found [here](https://docs.deepmodeling.com/projects/deepmd/en/v3.1.0rc0/train/train-input.html#argument:model[standard]/descriptor[dpa3]/repflow).

For demonstration simplicity, **we will use only 200 training steps in this example**:

In [5]:
!cd /deepmd-kit/examples/water/dpa3 && sed -i "s/\"numb_steps\": 1000000/\"numb_steps\": 200/g" input_torch.json

Note that optimal performance typically requires around **1 million steps or more** in real-world scenarios.

We then run training:

In [6]:
!cd /deepmd-kit/examples/water/dpa3 && dp --pt train input_torch.json --skip-neighbor-stat

To get the best performance, it is recommended to adjust the number of threads by setting the environment variables OMP_NUM_THREADS, DP_INTRA_OP_PARALLELISM_THREADS, and DP_INTER_OP_PARALLELISM_THREADS. See https://deepmd.rtfd.io/parallelism/ for more information.
[2025-06-10 15:25:17,994] DEEPMD INFO    DeePMD version: 3.1.0rc0
[2025-06-10 15:25:17,994] DEEPMD INFO    Configuration path: input_torch.json
[bohrium-150-1334359:00768] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.bohrium-150-1334359.0/jf.0/1226899456/shared_mem_cuda_pool.bohrium-150-1334359 could be created.
[bohrium-150-1334359:00768] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 
[2025-06-10 15:25:19,277] DEEPMD INFO     _____               _____   __  __  _____           _     _  _   
[2025-06-10 15:25:19,277] DEEPMD INFO    |  __ \             |  __ \ |  \/  ||  __ \         | |   (_)| |  
[2025-06-10 15:25:19,277] DEEPMD INFO    | |  | |  _

Now we get a toy DPA3 model for just demonstration.

## 3. Finetune from the pretrained DPA3 model

Next, we provide DPA3 models pretrained on [**OpenLAM-v1 datasets**](https://www.aissquare.com/datasets/detail?pageType=datasets&name=OpenLAM-TrainingSet-v1&id=308), including application-oriented datasets such as Alloy, Semiconductor, and Drug-like molecules, as well as open source DFT datasets like OMat, MPtraj, and OC20.

First we copy two pretrained DPA3 models (3 layers and 9 layers) from mounted Bohrium dataset:

In [7]:
!cp -r /bohr/dpa3-model-d4ju/v2 ./dpa3-model && tree ./dpa3-model -L 1

./dpa3-model
├── 0211_prod_9wide_300w.pt
├── 0217_l3_300w.pt
├── 3wide_input.json
├── 9wide_input.json
├── DPA-3.1-3M.pt
└── input_pretrain_3.1_3M.json

0 directories, 6 files


This directory contains the following main files:
1. **DPA-3.1-3M.pt** : multi-task pretrained DPA3-L16 model on OpenLAM-v1 datasets, downloaded from [AIS-Square](https://www.aissquare.com/models/detail?pageType=models&name=DPA-3.1-3M&id=343).
2. **input_pretrain_3.1_3M.json** : the corresponding pretrained input for **DPA-3.1-3M.pt**.

3. Other files in preview edition: **0211_prod_9wide_300w.pt**, **0217_l3_300w.pt** are different DPA3 variants also pre-trained on OpenLAM-v1 datasets, with corresponding inputs **9wide_input.json** and **3wide_input.json**.

We will demonstrate fine-tuning using **DPA-3.1-3M.pt** as an example. 
Readers can also apply identical procedures to other variants.

First we will prepare the input for finetuning:

In [8]:
!cd /deepmd-kit/examples/water && mkdir -p dpa3_finetune && cp /dpa3-model/DPA-3.1-3M.pt ./dpa3_finetune && cp /dpa3-model/input_pretrain_3.1_3M.json ./dpa3_finetune && cp dpa3/input_torch.json ./dpa3_finetune && tree /deepmd-kit/examples/water/dpa3_finetune -L 1

/deepmd-kit/examples/water/dpa3_finetune
├── DPA-3.1-3M.pt
├── input_pretrain_3.1_3M.json
└── input_torch.json

0 directories, 3 files


When performing fine-tuning based on pretrained models, it is crucial to **maintain identical model configurations (`"model"` scope in json file) between the pretraining and fine-tuning phases**. 

The model configuration may differ from our previous from-scratch training examples, therefore we will perform the following replacement to ensure the same model section in the finetune input:

In [9]:
import json

## Load pretrained model input
pretrained_input = json.load(open('/deepmd-kit/examples/water/dpa3_finetune/input_pretrain_3.1_3M.json', 'r'))

## Load example input
example_input = json.load(open('/deepmd-kit/examples/water/dpa3_finetune/input_torch.json', 'r'))

## Get pretrained model configuration
dpa3_descriptor = pretrained_input['model']['shared_dict']['dpa3_descriptor']
fitting = pretrained_input['model']['shared_dict']['shared_fitting']

## Do model configuration replacement
example_input['model']['descriptor'] = dpa3_descriptor
example_input['model']['fitting_net'] = fitting

# (Optional) set a smaller start learning rate
example_input['learning_rate']['start_lr'] = 1e-4

## Save finetune input
json.dump(example_input, open('/deepmd-kit/examples/water/dpa3_finetune/input_finetune.json', 'w'), indent=4)

In [10]:
!cd /deepmd-kit/examples/water/dpa3_finetune && cat input_finetune.json

{
    "_comment": "that's all",
    "model": {
        "type_map": [
            "O",
            "H"
        ],
        "descriptor": {
            "type": "dpa3",
            "repflow": {
                "n_dim": 128,
                "e_dim": 64,
                "a_dim": 32,
                "nlayers": 16,
                "e_rcut": 6.0,
                "e_rcut_smth": 5.3,
                "e_sel": 1200,
                "a_rcut": 4.0,
                "a_rcut_smth": 3.5,
                "a_sel": 300,
                "axis_neuron": 4,
                "skip_stat": true,
                "a_compress_rate": 1,
                "a_compress_e_rate": 2,
                "a_compress_use_split": true,
                "update_angle": true,
                "smooth_edge_update": true,
                "use_dynamic_sel": true,
                "sel_reduce_factor": 10.0,
                "use_exp_switch": true,
                "update_style": "res_residual",
                "update_residual": 0.1,
         

Then, note that the pretrained model is pretrained in a multi-task manner, featuring a unified backbone (referred to as the unified descriptor) and several fitting nets for different datasets. For detailed information, refer to the [DPA2 paper](https://www.nature.com/articles/s41524-024-01493-2).

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/b46229d6746e4358be3181363a158efb/94935036-1adb-4562-ae45-b7839e8666ae.png)

Before finetuning, we need to **select a specific fitting net** from the model to make predictions. To list the available fitting nets (model-branch) in this pretrained model, use the following command:

In [11]:
!cd /deepmd-kit/examples/water/dpa3_finetune && dp --pt show DPA-3.1-3M.pt model-branch

[2025-06-10 15:28:32,839] DEEPMD WARNING To get the best performance, it is recommended to adjust the number of threads by setting the environment variables OMP_NUM_THREADS, DP_INTRA_OP_PARALLELISM_THREADS, and DP_INTER_OP_PARALLELISM_THREADS. See https://deepmd.rtfd.io/parallelism/ for more information.
[2025-06-10 15:28:39,354] DEEPMD INFO    This is a multitask model
[2025-06-10 15:28:39,355] DEEPMD INFO    Available model branches are ['Domains_Alloy', 'Domains_Anode', 'Domains_Cluster', 'Domains_Drug', 'Domains_FerroEle', 'Domains_SSE_PBE', 'Domains_SemiCond', 'H2O_H2O_PD', 'Metals_AlMgCu', 'Metals_Sn', 'Metals_Ti', 'Metals_V', 'Metals_W', 'Others_HfO2', 'Domains_SSE_PBESol', 'Domains_Transition1x', 'Metals_AgAu_PBED3', 'Others_In2Se3', 'MP_traj_v024_alldata_mixu', 'Alloy_tongqi', 'SSE_ABACUS', 'Hybrid_Perovskite', 'solvated_protein_fragments', 'Electrolyte', 'ODAC23', 'Alex2D', 'Omat24', 'SPICE2', 'OC20M', 'OC22', 'Organic_Reactions', 'RANDOM'], where 'RANDOM' means using a rando

This lists all the available fitting nets. Then you need to select a fitting net that closely matches your system, here we take `MP_traj_v024_alldata_mixu`, which stands for branch of MPtraj dataset for example.

In [12]:
!cd /deepmd-kit/examples/water/dpa3_finetune && dp --pt train input_finetune.json --skip-neighbor-stat --finetune DPA-3.1-3M.pt --model-branch MP_traj_v024_alldata_mixu

To get the best performance, it is recommended to adjust the number of threads by setting the environment variables OMP_NUM_THREADS, DP_INTRA_OP_PARALLELISM_THREADS, and DP_INTER_OP_PARALLELISM_THREADS. See https://deepmd.rtfd.io/parallelism/ for more information.
[2025-06-10 15:28:50,552] DEEPMD INFO    DeePMD version: 3.1.0rc0
[2025-06-10 15:28:50,552] DEEPMD INFO    Configuration path: input_finetune.json
[bohrium-150-1334359:00892] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.bohrium-150-1334359.0/jf.0/1618870272/shared_mem_cuda_pool.bohrium-150-1334359 could be created.
[bohrium-150-1334359:00892] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 
[2025-06-10 15:28:51,781] DEEPMD INFO     _____               _____   __  __  _____           _     _  _   
[2025-06-10 15:28:51,781] DEEPMD INFO    |  __ \             |  __ \ |  \/  ||  __ \         | |   (_)| |  
[2025-06-10 15:28:51,781] DEEPMD INFO    | |  | |

Then finally we have a finetuned DPA3 model.

## 4. Test models

After completing training/finetuning, we typically evaluate model performance by examining accuracy across different material systems.

First we can compare the loss curves (lcurve.out) during training/finetuning:

In [13]:
!head /deepmd-kit/examples/water/dpa3/lcurve.out

# step       rmse_val    rmse_trn    rmse_e_val  rmse_e_trn    rmse_f_val  rmse_f_trn    rmse_v_val  rmse_v_trn   lr      
# If there is no available reference data, rmse_*_{val,trn} will print nan
      1      9.38e+00    1.05e+01      5.02e-01    1.11e+00      8.85e-01    7.96e-01           nan         nan    1.0e-03
    100      6.67e+00    6.69e+00      8.85e-03    5.71e-03      8.13e-01    8.16e-01           nan         nan    1.8e-04
    200      6.01e+00    6.63e+00      7.27e-03    1.21e-02      7.65e-01    8.42e-01           nan         nan    3.1e-05


In [14]:
!head /deepmd-kit/examples/water/dpa3_finetune/lcurve.out

# step       rmse_val    rmse_trn    rmse_e_val  rmse_e_trn    rmse_f_val  rmse_f_trn    rmse_v_val  rmse_v_trn   lr      
# If there is no available reference data, rmse_*_{val,trn} will print nan
      1      2.44e+00    3.09e+00      2.63e-03    6.05e-04      2.44e-01    3.09e-01           nan         nan    1.0e-04
    100      1.75e-01    1.68e-01      2.53e-03    1.88e-03      1.54e-02    1.64e-02           nan         nan    5.5e-05
    200      1.39e-01    1.32e-01      5.04e-04    5.18e-04      1.60e-02    1.52e-02           nan         nan    3.0e-05


Then we will typically run inference directly on target systems using the `dp test` command to measure prediction errors, which provides more precise accuracy evaluation:

In [15]:
!dp test --help

usage: dp test [-h] [-v {DEBUG,3,INFO,2,WARNING,1,ERROR,0}] [-l LOG_PATH]
               [-m MODEL] [-s SYSTEM | -f DATAFILE] [-S SET_PREFIX]
               [-n NUMB_TEST] [-r RAND_SEED] [--shuffle-test] [-d DETAIL_FILE]
               [-a] [--head HEAD]

options:
  -h, --help            show this help message and exit
  -v {DEBUG,3,INFO,2,WARNING,1,ERROR,0}, --log-level {DEBUG,3,INFO,2,WARNING,1,ERROR,0}
                        set verbosity level by string or number, 0=ERROR, 1=WARNING, 2=INFO and 3=DEBUG (default: INFO)
  -l LOG_PATH, --log-path LOG_PATH
                        set log file to log messages to disk, if not specified, the logs will only be output to console (default: None)
  -m MODEL, --model MODEL
                        Frozen model file (prefix) to import. TensorFlow backend: suffix is .pb; PyTorch backend: suffix is .pth. (default: frozen_model)
  -s SYSTEM, --system SYSTEM
                        The system dir. Recursively detect systems in this directory (defau

In [16]:
!dp --pt test -m /deepmd-kit/examples/water/dpa3/model.ckpt.pt -s /deepmd-kit/examples/water/data/data_3

[2025-06-10 15:30:44,362] DEEPMD WARNING To get the best performance, it is recommended to adjust the number of threads by setting the environment variables OMP_NUM_THREADS, DP_INTRA_OP_PARALLELISM_THREADS, and DP_INTER_OP_PARALLELISM_THREADS. See https://deepmd.rtfd.io/parallelism/ for more information.
[2025-06-10 15:30:47,236] DEEPMD INFO    # ---------------output of dp test--------------- 
[2025-06-10 15:30:47,237] DEEPMD INFO    # testing system : /deepmd-kit/examples/water/data/data_3
[2025-06-10 15:30:52,129] DEEPMD INFO    Adjust batch size from 1024 to 2048
[2025-06-10 15:30:56,996] DEEPMD INFO    Adjust batch size from 2048 to 4096
[2025-06-10 15:31:01,260] DEEPMD INFO    Adjust batch size from 4096 to 2048
[2025-06-10 15:31:11,222] DEEPMD INFO    # number of test data : 80 
[2025-06-10 15:31:11,222] DEEPMD INFO    Energy MAE         : 1.535458e+00 eV
[2025-06-10 15:31:11,223] DEEPMD INFO    Energy RMSE        : 1.670812e+00 eV
[2025-06-10 15:31:11,223] DEEPMD INFO    Energy

In [17]:
!dp --pt test -m /deepmd-kit/examples/water/dpa3_finetune/model.ckpt.pt -s /deepmd-kit/examples/water/data/data_3

[2025-06-10 15:31:21,767] DEEPMD WARNING To get the best performance, it is recommended to adjust the number of threads by setting the environment variables OMP_NUM_THREADS, DP_INTRA_OP_PARALLELISM_THREADS, and DP_INTER_OP_PARALLELISM_THREADS. See https://deepmd.rtfd.io/parallelism/ for more information.
[2025-06-10 15:31:26,568] DEEPMD INFO    # ---------------output of dp test--------------- 
[2025-06-10 15:31:26,568] DEEPMD INFO    # testing system : /deepmd-kit/examples/water/data/data_3
[2025-06-10 15:31:38,204] DEEPMD INFO    Adjust batch size from 1024 to 2048
[2025-06-10 15:31:50,225] DEEPMD INFO    Adjust batch size from 2048 to 1024
[2025-06-10 15:32:49,767] DEEPMD INFO    # number of test data : 80 
[2025-06-10 15:32:49,767] DEEPMD INFO    Energy MAE         : 1.426611e-01 eV
[2025-06-10 15:32:49,767] DEEPMD INFO    Energy RMSE        : 1.468096e-01 eV
[2025-06-10 15:32:49,767] DEEPMD INFO    Energy MAE/Natoms  : 7.430266e-04 eV
[2025-06-10 15:32:49,767] DEEPMD INFO    Energ

We can see that the fine-tuned model demonstrates significantly better accuracy (benefiting from the pre-trained model).

## 5. Python interface for MD

Finally, we demonstrate how to use trained models for **practical simulations in Python via the ASE interface**. 

In [18]:
!pip install ase

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.3 MB/s eta 0:00:00a 0:00:01


We need first freeze out checkout file into one `pth` frozen model:

In [19]:
!cd /deepmd-kit/examples/water/dpa3_finetune && dp --pt freeze -o dpa3_water_finetune.pth

To get the best performance, it is recommended to adjust the number of threads by setting the environment variables OMP_NUM_THREADS, DP_INTRA_OP_PARALLELISM_THREADS, and DP_INTER_OP_PARALLELISM_THREADS. See https://deepmd.rtfd.io/parallelism/ for more information.
[2025-06-10 15:33:15,425] DEEPMD INFO    DeePMD version: 3.1.0rc0
[2025-06-10 15:33:17,886] DEEPMD INFO    Saved frozen model to dpa3_water_finetune.pth


We can also directly freeze the pretrained model without further finetuning to perform MD: 

```
dp --pt freeze -c DPA-3.1-3M.pt -o dpa3_water_branch.pth --model-branch H2O_H2O_PD

```



Then we can run the following Python code for prediction or optimization:

In [20]:
%%writefile ase_md.py
from ase import Atoms
from deepmd.calculator import DP

## Compute potential energy
dp = DP("/deepmd-kit/examples/water/dpa3_finetune/dpa3_water_finetune.pth")
water = Atoms('H2O', positions=[(0.7601, 1.9270, 1), (1.9575, 1, 1), (1., 1., 1.)], cell=[100, 100, 100])
water.calc = dp
print(water.get_potential_energy())
print(water.get_forces())

## Run BFGS structure optimization
from ase.optimize import BFGS
dyn = BFGS(water)
dyn.run(fmax=1e-3)
print(water.get_positions())

Writing ase_md.py


In [21]:
!python ase_md.py

To get the best performance, it is recommended to adjust the number of threads by setting the environment variables OMP_NUM_THREADS, DP_INTRA_OP_PARALLELISM_THREADS, and DP_INTER_OP_PARALLELISM_THREADS. See https://deepmd.rtfd.io/parallelism/ for more information.
-467.41012290392746
[[-0.1189101   0.24367994  0.        ]
 [ 0.26781094 -0.05406877  0.        ]
 [-0.14890084 -0.18961118  0.        ]]
      Step     Time          Energy          fmax
BFGS:    0 15:33:43     -467.410123        0.273214
BFGS:    1 15:33:55     -467.411279        0.124385
BFGS:    2 15:34:15     -467.411456        0.048061
BFGS:    3 15:34:15     -467.411587        0.034236
BFGS:    4 15:34:15     -467.411628        0.014943
BFGS:    5 15:34:15     -467.411632        0.003338
BFGS:    6 15:34:15     -467.411632        0.000662
[[0.75499215 1.93081114 1.        ]
 [1.96247996 0.9960098  1.        ]
 [1.00012789 1.00017906 1.        ]]


### Ref
1. [DPA3 paper](https://arxiv.org/abs/2506.01686)
2. [LAMBench paper](https://arxiv.org/abs/2504.19578)
2. [DPA2 paper](https://www.nature.com/articles/s41524-024-01278-7)
2. [DPA2 finetune tutorial](https://www.aissquare.com/models/detail?pageType=models&name=DPA-2.3.1-v3.0.0rc0&id=287)
3. [AIS-Square](https://www.aissquare.com)
4. [OpenLAM Initiative](https://www.aissquare.com/openlam)
5. [LAMBench Website](https://www.aissquare.com/openlam?tab=Benchmark)